CNN 예제

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
trainlist, testlist = [], []
#학습 데이터 로드를 위한 정의
#각 파일에는 학습데이터의 경로와 LABEL이 들어있음
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        #rainlist.append([tmp[0].replace("\\","/"), tmp[1]]) 리눅스버전
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        #testlist.append([tmp[0].replace("\\","/"), tmp[1]])
        testlist.append([tmp[0], tmp[1]])

In [4]:
#이미지 해상도 지정
IMG_H = 100
IMG_W = 100
IMG_C = 3
#각 파일을 읽어 오기 위한 함수
def readimg(path):
    img = plt.imread(path)
    return img
#전체 이미지를 불러오기 위한 함수
def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [6]:
#분류해야 하는 클래스수
num_class = 3 

#그래프를 활용한 모델 구성
with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C])
    Y = tf.placeholder(tf.int32, [None])
    keep_prob = tf.placeholder(tf.float32)
    
    with tf.variable_scope('CNN'):
        L1 = tf.layers.conv2d(X, 20, 3, (2, 2), padding='same', activation=tf.nn.relu)
        L1 = tf.layers.max_pooling2d(L1, 2, 2)
        L2 = tf.layers.conv2d(L1, 40, 3, (2, 2), padding='same', activation=tf.nn.relu)
        L2 = tf.layers.flatten(L2)
     
        out = tf.layers.dense(L2, num_class)
        
    with tf.variable_scope('Loss'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=out))
        
    train = tf.train.AdamOptimizer(1e-3).minimize(loss)
    saver = tf.train.Saver()

In [7]:
batch_size = 1461
total_cost = 0
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(30):
        batch_data, batch_label = batch(trainlist, batch_size)
        _, cost_val = sess.run([train, loss], feed_dict = {X: batch_data, Y: batch_label,keep_prob: 0.8})
        total_cost += cost_val
        print('Epoch : ',i+1, 'Avg. cost =', '{:.3f}'.format(total_cost / batch_size))
        
    saver.save(sess, 'logs/model.ckpt', global_step = i+1)
    
print('최적화 완료!')

Epoch :  1 Avg. cost = 0.025
Epoch :  2 Avg. cost = 0.111
Epoch :  3 Avg. cost = 0.234
Epoch :  4 Avg. cost = 0.299
Epoch :  5 Avg. cost = 0.306
Epoch :  6 Avg. cost = 0.316
Epoch :  7 Avg. cost = 0.336
Epoch :  8 Avg. cost = 0.356
Epoch :  9 Avg. cost = 0.375
Epoch :  10 Avg. cost = 0.390
Epoch :  11 Avg. cost = 0.398
Epoch :  12 Avg. cost = 0.400
Epoch :  13 Avg. cost = 0.404
Epoch :  14 Avg. cost = 0.408
Epoch :  15 Avg. cost = 0.410
Epoch :  16 Avg. cost = 0.410
Epoch :  17 Avg. cost = 0.410
Epoch :  18 Avg. cost = 0.411
Epoch :  19 Avg. cost = 0.412
Epoch :  20 Avg. cost = 0.413
Epoch :  21 Avg. cost = 0.413
Epoch :  22 Avg. cost = 0.413
Epoch :  23 Avg. cost = 0.413
Epoch :  24 Avg. cost = 0.414
Epoch :  25 Avg. cost = 0.414
Epoch :  26 Avg. cost = 0.414
Epoch :  27 Avg. cost = 0.414
Epoch :  28 Avg. cost = 0.414
Epoch :  29 Avg. cost = 0.414
Epoch :  30 Avg. cost = 0.414
최적화 완료!


In [9]:
acc = 0
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(out, feed_dict = {X:batch_data,keep_prob: 1})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    print('정확도:',acc/len(testlist))

INFO:tensorflow:Restoring parameters from logs\model.ckpt-30
[ 38.420643 -57.33401   41.42887 ] 0
[ 35.56878 -66.0061   45.23953] 0
[ 36.61056 -66.35453  45.47446] 0
[ 37.559883 -67.9472    45.140163] 0
[ 36.61303 -67.61944  44.42542] 0
[ 35.24984  -66.38593   46.838074] 0
[ 32.939423 -66.534485  46.348934] 0
[ 36.763714 -62.247227  38.99051 ] 0
[ 35.799706 -63.241833  40.468536] 0
[ 36.103546 -64.90862   40.44495 ] 0
[ 35.98736 -65.06871  42.63256] 0
[ 33.847286 -64.813156  45.255543] 0
[ 31.295128 -64.10521   45.99771 ] 0
[ 31.872847 -63.8883    45.311104] 0
[ 30.542593 -62.918396  45.65383 ] 0
[ 29.874802 -64.10076   45.50418 ] 0
[ 28.566805 -59.07056   44.643177] 0
[ 27.082245 -59.20379   46.062546] 0
[ 28.900448 -58.201633  43.63957 ] 0
[ 28.66646  -58.02627   46.049866] 0
[ 27.435928 -57.32667   45.08872 ] 0
[ 29.25932  -56.87889   45.622513] 0
[ 28.723404 -57.658314  45.59806 ] 0
[ 28.667734 -53.289818  39.350994] 0
[ 28.875385 -53.951077  46.818943] 0
[ 26.808098 -53.07273   46

RNN 으로 이미지 분류

In [10]:
# 머신러닝 학습의 Hello World 와 같은 MNIST(손글씨 숫자 인식) 문제를 신경망으로 풀어봅니다.
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

#########
# 옵션 설정
######
learning_rate = 0.001
total_epoch = 30
batch_size = 128

# RNN 은 순서가 있는 자료를 다루므로,
# 한 번에 입력받는 갯수와, 총 몇 단계로 이루어져있는 데이터를 받을지를 설정해야합니다.
# 이를 위해 가로 픽셀수를 n_input 으로, 세로 픽셀수를 입력 단계인 n_step 으로 설정하였습니다.
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [11]:
#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [12]:
# RNN 에 학습에 사용할 셀을 생성합니다
# BasicRNNCell,BasicLSTMCell,GRUCell
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

# RNN 신경망을 생성합니다
# 원래는 다음과 같은 과정을 거쳐야 하지만
# states = tf.zeros(batch_size)
# for i in range(n_step):
#     outputs, states = cell(X[[:, i]], states)
# ...
# 다음처럼 tf.nn.dynamic_rnn 함수를 사용하면
# CNN 의 tf.nn.conv2d 함수처럼 간단하게 RNN 신경망을 만들어줍니다.
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [13]:
# 결과를 Y의 다음 형식과 바꿔야 하기 때문에
# Y : [batch_size, n_class]
# outputs 의 형태를 이에 맞춰 변경해야합니다.
# outputs : [batch_size, n_step, n_hidden]
#        -> [n_step, batch_size, n_hidden]
#        -> [batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [14]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(total_epoch):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # X 데이터를 RNN 입력 데이터에 맞게 [batch_size, n_step, n_input] 형태로 변환합니다.
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.531
Epoch: 0002 Avg. cost = 0.236
Epoch: 0003 Avg. cost = 0.177
Epoch: 0004 Avg. cost = 0.144
Epoch: 0005 Avg. cost = 0.134
Epoch: 0006 Avg. cost = 0.121
Epoch: 0007 Avg. cost = 0.113
Epoch: 0008 Avg. cost = 0.109
Epoch: 0009 Avg. cost = 0.100
Epoch: 0010 Avg. cost = 0.094
Epoch: 0011 Avg. cost = 0.095
Epoch: 0012 Avg. cost = 0.089
Epoch: 0013 Avg. cost = 0.086
Epoch: 0014 Avg. cost = 0.090
Epoch: 0015 Avg. cost = 0.081
Epoch: 0016 Avg. cost = 0.080
Epoch: 0017 Avg. cost = 0.077
Epoch: 0018 Avg. cost = 0.075
Epoch: 0019 Avg. cost = 0.075
Epoch: 0020 Avg. cost = 0.070
Epoch: 0021 Avg. cost = 0.069
Epoch: 0022 Avg. cost = 0.067
Epoch: 0023 Avg. cost = 0.076
Epoch: 0024 Avg. cost = 0.063
Epoch: 0025 Avg. cost = 0.062
Epoch: 0026 Avg. cost = 0.068
Epoch: 0027 Avg. cost = 0.068
Epoch: 0028 Avg. cost = 0.060
Epoch: 0029 Avg. cost = 0.060
Epoch: 0030 Avg. cost = 0.065
최적화 완료!


In [15]:
#########
# 결과 확인
######
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy,
                       feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.9724


자연어 처리

In [16]:
# 자연어 처리나 음성 처리 분야에 많이 사용되는 RNN 의 기본적인 사용법을 익힙니다.
# 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성하는 프로그램입니다.
import tensorflow as tf
import numpy as np


char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# one-hot 인코딩 사용 및 디코딩을 하기 위해 연관 배열을 만듭니다.
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
# 다음 배열은 입력값과 출력값으로 다음처럼 사용할 것 입니다.
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [17]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # 여기서 생성하는 input_batch 와 target_batch 는
        # 알파벳 배열의 인덱스 번호 입니다.
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [num_dic[n] for n in seq[:-1]]
        # 3, 3, 15, 4, 3 ...
        target = num_dic[seq[-1]]
        # one-hot 인코딩을 합니다.
        # if input is [0, 1, 2]:
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        input_batch.append(np.eye(dic_len)[input])
        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
        # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
        # 이 예제에서 사용할 손실 함수인 sparse_softmax_cross_entropy_with_logits 는
        # one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨주면 됩니다.
        target_batch.append(target)

    return input_batch, target_batch

In [18]:
#########
# 옵션 설정
######
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
# 타입 스텝: [1 2 3] => 3
# RNN 을 구성하는 시퀀스의 갯수입니다.
n_step = 3
# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개가 됩니다.
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
# 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류합니다.
n_input = n_class = dic_len

신경망 모델 구성

In [19]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
# 비용함수에 sparse_softmax_cross_entropy_with_logits 을 사용하므로
# 출력값과의 계산을 위한 원본값의 형태는 one-hot vector가 아니라 인덱스 숫자를 그대로 사용하기 때문에
# 다음처럼 하나의 값만 있는 1차원 배열을 입력값으로 받습니다.
# [3] [3] [15] [4] ...
# 기존처럼 one-hot 인코딩을 사용한다면 입력값의 형태는 [None, n_class] 여야합니다.
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [20]:
# RNN 셀을 생성합니다.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합 방지를 위한 Dropout 기법을 사용합니다.
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
# 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성합니다.
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 여러개의 셀을 조합한 RNN 셀을 생성합니다.
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_rnn 함수를 이용해 순환 신경망을 만듭니다.
# time_major=True
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
# 최종 결과는 one-hot 인코딩 형식으로 만듭니다
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [22]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 4.352989
Epoch: 0002 cost = 3.397109
Epoch: 0003 cost = 1.989659
Epoch: 0004 cost = 1.243571
Epoch: 0005 cost = 0.815087
Epoch: 0006 cost = 0.509394
Epoch: 0007 cost = 0.491872
Epoch: 0008 cost = 0.377882
Epoch: 0009 cost = 0.284965
Epoch: 0010 cost = 0.298327
Epoch: 0011 cost = 0.337557
Epoch: 0012 cost = 0.234524
Epoch: 0013 cost = 0.336819
Epoch: 0014 cost = 0.463205
Epoch: 0015 cost = 0.103225
Epoch: 0016 cost = 0.224904
Epoch: 0017 cost = 0.324641
Epoch: 0018 cost = 0.080399
Epoch: 0019 cost = 0.122818
Epoch: 0020 cost = 0.053106
Epoch: 0021 cost = 0.151145
Epoch: 0022 cost = 0.219619
Epoch: 0023 cost = 0.049309
Epoch: 0024 cost = 0.102859
Epoch: 0025 cost = 0.022402
Epoch: 0026 cost = 0.016609
Epoch: 0027 cost = 0.034770
Epoch: 0028 cost = 0.022338
Epoch: 0029 cost = 0.019497
Epoch: 0030 cost = 0.011235
최적화 완료!


결과확인

In [23]:
# 레이블값이 정수이므로 예측값도 정수로 변경해줍니다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
# one-hot 인코딩이 아니므로 입력값을 그대로 비교합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []

In [24]:
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


Seq2Seq

In [25]:
# 챗봇, 번역, 이미지 캡셔닝등에 사용되는 시퀀스 학습/생성 모델인 Seq2Seq 을 구현해봅니다.
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
import tensorflow as tf
import numpy as np

# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd']
#       to   -> ['t', 'o', 'P', 'P']
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

In [26]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [27]:
#########
# 옵션 설정
######
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = n_input = dic_len

In [28]:
#########
# 신경망 모델 구성
######
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])

In [29]:
# 인코더 셀을 구성한다.
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)

In [30]:
# 디코더 셀을 구성한다.
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)


model = tf.layers.dense(outputs, n_class, activation=None)


cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [31]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 3.766512
Epoch: 0002 cost = 2.630184
Epoch: 0003 cost = 1.669039
Epoch: 0004 cost = 1.161018
Epoch: 0005 cost = 0.760549
Epoch: 0006 cost = 0.479378
Epoch: 0007 cost = 0.322109
Epoch: 0008 cost = 0.265935
Epoch: 0009 cost = 0.267513
Epoch: 0010 cost = 0.106740
Epoch: 0011 cost = 0.130364
Epoch: 0012 cost = 0.035825
Epoch: 0013 cost = 0.185538
Epoch: 0014 cost = 0.137094
Epoch: 0015 cost = 0.038254
Epoch: 0016 cost = 0.203144
Epoch: 0017 cost = 0.022944
Epoch: 0018 cost = 0.015545
Epoch: 0019 cost = 0.009953
Epoch: 0020 cost = 0.011000
Epoch: 0021 cost = 0.049217
Epoch: 0022 cost = 0.031701
Epoch: 0023 cost = 0.010337
Epoch: 0024 cost = 0.004422
Epoch: 0025 cost = 0.003346
Epoch: 0026 cost = 0.003673
Epoch: 0027 cost = 0.005714
Epoch: 0028 cost = 0.005086
Epoch: 0029 cost = 0.003273
Epoch: 0030 cost = 0.003427
Epoch: 0031 cost = 0.001901
Epoch: 0032 cost = 0.005482
Epoch: 0033 cost = 0.011603
Epoch: 0034 cost = 0.015925
Epoch: 0035 cost = 0.005092
Epoch: 0036 cost = 0

In [32]:
#########
# 번역 테스트
######
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

In [33]:
print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))



=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 사랑
